In [5]:
import json
import codecs
import csv
import pandas as pd

NEO_VER_CSV = 'data/out/neo_ver.csv'
NEO_OUT_JSON = 'data/input/records.json'
df = pd.read_csv(NEO_VER_CSV)

def get_csv_data(file_path):
    with open(file_path) as f:
        data = csv.reader(f)
    return data

jsonData = json.load(codecs.open('data/input/records.json', 'r', 'utf-8-sig'))
# csv_reader = csv.reader(NEO_VER_CSV, delimiter=",")


In [6]:
df2 = df[df['pkg_name'] == 'glib']
df2

,:ID,pkg_name,pkgman,version,:LABEL,homepage,repo
192520,version17029cargo,glib,cargo,0.0.5,Version,https://github.com/gtk-rs/gtk-rs-core,https://developer.gnome.org/glib
192521,version177941cargo,glib,cargo,0.8.2,Version,https://github.com/gtk-rs/gtk-rs-core,https://gtk-rs.org/
192522,version135106cargo,glib,cargo,0.7.0,Version,https://github.com/gtk-rs/gtk-rs-core,https://gtk-rs.org/
192523,version63643cargo,glib,cargo,0.3.1,Version,https://github.com/gtk-rs/gtk-rs-core,https://gtk-rs.org/
192524,version18630cargo,glib,cargo,0.0.7,Version,https://github.com/gtk-rs/gtk-rs-core,https://gtk-rs.org/
...,...,...,...,...,...,...,...
17316106,version16588286npm,glib,npm,1.0.2,Version,git://github.com/bleupen/hapi-logger.git,https://github.com/bleupen/hapi-logger
17316107,version16588287npm,glib,npm,1.1.0,Version,git://github.com/bleupen/hapi-logger.git,https://github.com/bleupen/hapi-logger
17316108,version16588288npm,glib,npm,1.2.0,Version,git://github.com/bleupen/hapi-logger.git,https://github.com/bleupen/hapi-logger
17316109,version16588289npm,glib,npm,1.3.0,Version,git://github.com/bleupen/hapi-logger.git,https://github.com/bleupen/hapi-logger


In [7]:
dfs = []
df2 = df[df['pkg_name'] == 'glib']
df2 = df2[~df2['repo'].isnull()]
pkgmans = df2['pkgman'].unique()
for pkgman in pkgmans:
    dfs.append(df2[df2['pkgman'] == pkgman].drop_duplicates('repo'))
master_df = pd.concat(dfs)
master_df
master_df[master_df.duplicated(subset='repo', keep=False)]

,:ID,pkg_name,pkgman,version,:LABEL,homepage,repo
192520,version17029cargo,glib,cargo,0.0.5,Version,https://github.com/gtk-rs/gtk-rs-core,https://developer.gnome.org/glib
534170,version205chromebrew,glib,chromebrew,2.72.1,Version,https://gitlab.gnome.org/GNOME/glib.git,https://developer.gnome.org/glib
709199,version1875homebrew,glib,homebrew,2.72.1,Version,https://download.gnome.org/sources/glib/2.72/g...,https://developer.gnome.org/glib/
25850731,version625vcpkg,glib,vcpkg,2.70.1,Version,NaN,https://developer.gnome.org/glib/


In [8]:
master_df[master_df.duplicated(subset='repo', keep=False)]

,:ID,pkg_name,pkgman,version,:LABEL,homepage,repo
192520,version17029cargo,glib,cargo,0.0.5,Version,https://github.com/gtk-rs/gtk-rs-core,https://developer.gnome.org/glib
534170,version205chromebrew,glib,chromebrew,2.72.1,Version,https://gitlab.gnome.org/GNOME/glib.git,https://developer.gnome.org/glib
709199,version1875homebrew,glib,homebrew,2.72.1,Version,https://download.gnome.org/sources/glib/2.72/g...,https://developer.gnome.org/glib/
25850731,version625vcpkg,glib,vcpkg,2.70.1,Version,NaN,https://developer.gnome.org/glib/


In [9]:

for df in dfs:
    
    pass

In [10]:
for row in jsonData:
    name = row["name"]
    pms_dfs = []
    for pkgman in row["pkgmans"]:
        df2 = df[df['pkg_name'] == name]
        df2 = df2[~df2['repo'].isnull()]
        df2_pkgman = df2['repo'].drop_duplicates()
        print(df2_pkgman)

        # df2 = df[df['pkg_name'] == name]
        # df2 = df2[~df2['repo'].isnull()]
        # uniqe_repos = df2['repo'].unique()
        # if len(uniqe_repos) == 1:
        #     print(name)
        #     print(pkgman)
        #     print()

Series([], Name: repo, dtype: object)
Series([], Name: repo, dtype: object)
